In [2]:
import numpy

In [3]:
def transform_to_base(RTo, Rc, Tc):
    translation2cam = RTo[:3, -1]
    rotation2cam = RTo[:3, :3]
    
    translation2base = Rc
    cam2base = Tc
    resR = cam2base @ rotation2cam
    resT = translation2cam + translation2base
    
    return resR, resT

In [4]:
import numpy as np
import math

def euler_to_rotation_matrix(roll, pitch, yaw):
    """
    Convert Euler angles (roll, pitch, yaw) to a rotation matrix.
    
    Parameters:
        roll:   Rotation around the x-axis (in radians)
        pitch:  Rotation around the y-axis (in radians)
        yaw:    Rotation around the z-axis (in radians)
        
    Returns:
        R:      3x3 rotation matrix
    """
    # Roll, pitch, and yaw angles
    cos_roll = math.cos(roll)
    sin_roll = math.sin(roll)
    cos_pitch = math.cos(pitch)
    sin_pitch = math.sin(pitch)
    cos_yaw = math.cos(yaw)
    sin_yaw = math.sin(yaw)

    # Compute rotation matrix
    R_roll = np.array([[1, 0, 0],
                       [0, cos_roll, -sin_roll],
                       [0, sin_roll, cos_roll]])
    
    R_pitch = np.array([[cos_pitch, 0, sin_pitch],
                        [0, 1, 0],
                        [-sin_pitch, 0, cos_pitch]])
    
    R_yaw = np.array([[cos_yaw, -sin_yaw, 0],
                      [sin_yaw, cos_yaw, 0],
                      [0, 0, 1]])

    # Combine the rotations
    R = np.dot(R_roll, np.dot(R_pitch, R_yaw))

    return R

# Example usage
# roll = np.radians(45)   # Example roll angle in radians
# pitch = np.radians(30)  # Example pitch angle in radians
# yaw = np.radians(60)    # Example yaw angle in radians

roll = np.array(1.6448)   # Example roll angle in radians
pitch = np.array(1.1151)  # Example pitch angle in radians
yaw = np.array(0.1311)    # Example yaw angle in radians


# Convert Euler angles to rotation matrix
rotation_matrix = euler_to_rotation_matrix(roll, pitch, yaw)

print("Rotation Matrix:")
print(rotation_matrix)


Rotation Matrix:
[[ 0.43631116 -0.05753037  0.8979548 ]
 [ 0.87814726 -0.19036534 -0.43888316]
 [ 0.19618858  0.98002617 -0.03253839]]


In [5]:
rotation_matrix[0, :] @ rotation_matrix[0, :]

1.0

In [6]:
rotation_matrix @ rotation_matrix.T

array([[ 1.00000000e+00,  0.00000000e+00, -6.93889390e-18],
       [ 0.00000000e+00,  1.00000000e+00, -1.04083409e-17],
       [-6.93889390e-18, -1.04083409e-17,  1.00000000e+00]])

In [7]:
rotation_matrix.T @ rotation_matrix

array([[ 1.00000000e+00, -2.77555756e-17, -6.93889390e-17],
       [-2.77555756e-17,  1.00000000e+00, -6.93889390e-18],
       [-6.93889390e-17, -6.93889390e-18,  1.00000000e+00]])

In [16]:
RT_h2b = np.array([[0.] * 4 for _ in range(4)])

In [19]:
rotation_matrix = euler_to_rotation_matrix(2.6659, -0.0365, -0.0966)
RT_h2b[:3, :3] = rotation_matrix
RT_h2b[:3, 3] = np.array([0.0710, -0.2413, 0.4115])
RT_h2b[3, 3] = 1
RT_h2b

array([[ 0.9946749 ,  0.09638559, -0.0364919 ,  0.071     ],
       [ 0.06910785, -0.88644302, -0.45764929, -0.2413    ],
       [-0.07645878,  0.45269038, -0.88838363,  0.4115    ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [20]:
RT_c2h = np.array([[0.] * 4 for _ in range(4)])

In [55]:
RT_c2h[:3, :3] = np.array([[-1, 0, 0], [0, -1, 0], [0, 0, 1]])
RT_c2h[:3, 3] = np.array([0.03, 0.065, -0.09])
RT_c2h[3, 3] = 1
RT_c2h

array([[-1.   ,  0.   ,  0.   ,  0.03 ],
       [ 0.   , -1.   ,  0.   ,  0.065],
       [ 0.   ,  0.   ,  1.   , -0.09 ],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])

In [56]:
RT_c2b = RT_h2b @ RT_c2h

In [57]:
import json
# Specify the filename
filename = "c2b-pose.json"

# Save the list representation of the array to a JSON file
with open(filename, 'w') as f:
    json.dump({'pose':RT_c2b.tolist()}, f)

In [47]:
with open(filename, 'r') as f:
    data = json.load(f)

In [48]:
RT_c2b = np.array(data['pose'])
RT_c2b

array([[-0.9946749 , -0.09638559, -0.0364919 ,  0.03190989],
       [-0.06910785,  0.88644302, -0.45764929, -0.23366334],
       [ 0.07645878, -0.45269038, -0.88838363,  0.3022359 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])